In [1]:
import torch
import numpy as np
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch import optim
from tqdm import tqdm 
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt
import time

D:\Program Files (x86)\Anaconda\envs\deeplearning\lib\site-packages\torchvision\io\image.py:11: UserWarning: Failed to load image Python extension: Could not find module 'D:\Program Files (x86)\Anaconda\envs\deeplearning\Lib\site-packages\torchvision\image.pyd' (or one of its dependencies). Try using the full path with constructor syntax.
  warn(f"Failed to load image Python extension: {e}")


In [2]:
def torchPCA(train, test, k):
    print(f"Performing PCA with k value: {k}")
    #FIT ON TRAIN, THEN TRANSFORM TRAIN *AND* TEST (i think?)
    #since batch size = entire dataset, just grab first batch from dataloader
    data = iter(train) 
    single_point = next(data) # in this case the 'single point' is the entire batch
    x = single_point[0]
    b, c, h, w = x.shape
    x = x.view(b, -1) # Flatten image
    x = x.numpy() # convert to np array for pca
    pca = PCA(n_components=k)
    pca.fit(x)
    x = pca.transform(x)
    x = torch.from_numpy(x).to(device)
    #TEST transform
    data = iter(test) 
    single_point = next(data)
    y = single_point[0]
    b, c, h, w = y.shape
    y = y.view(b, -1) # Flatten image
    y = y.numpy() # convert to np array for pca
    y = pca.transform(y) #transform according to train set for consistency (?)
    y = torch.from_numpy(y).to(device)
    return x, y

In [3]:
def trainPCA(train_data, trainX_PCA, model):    
    print("Training Start")
    start_time = time.time()
    model.train()
    loss_history = []
    loss = torch.Tensor([0])
    for epoch in tqdm(range(nepochs),
                      desc=f"Epoch",
                      unit="epoch",
                      disable=False):
        for (data, label) in tqdm(train_data,
                                  desc="iteration",
                                  unit="%",
                                  disable=True):
            optimizer.zero_grad(set_to_none=True) 

            # data = data.to(device)
            label = label.to(device)
            out = model(trainX_PCA) 

            loss = LossFunction(out, label)

            loss.backward() 
            optimizer.step()
            loss_history.append(loss.item())
        print(f"Epoch {epoch}: loss: {loss.item()}")
    elapsed = time.time() - start_time
    return elapsed, loss_history

In [4]:
def testPCA(test_data, testX_PCA, model):
    print("Test Start")
    model.eval()
    accuracy = 0
    for (data, label) in test_data:
        #data = data.to(device)
        label = label.to(device)
        out = model(testX_PCA)
        answers = out.max(dim=1)[1]
        accuracy += (answers == label).sum()
    print(f"Total accuracy = {accuracy / len(cifar10_test_dataset)*100:.2f}%")
    accuracy = (accuracy / len(cifar10_test_dataset)*100)
    return accuracy

In [5]:
def trainAndTest_PCA(k, train, test, model):
    trainX, testX = torchPCA(train, test, k)
    elapsed, loss_history = trainPCA(train, trainX, model)
    accuracy = testPCA(test, testX, model)
    return elapsed, loss_history, accuracy

In [6]:
class SimpleNet(nn.Module):
    def __init__(self,
                 ninputs=28*28,
                 nhidden=512,
                 nout=10,
                ):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(ninputs, nhidden),
            nn.ReLU(),
            nn.Linear(nhidden, nhidden),
            nn.ReLU(),
            nn.Linear(nhidden, nout),
            nn.ReLU(),
        )
        
    def forward(self, x):
        x = self.net(x)
        return x

In [7]:
#data
#load entire dataset for PCA (gaming PC go brr)
train_all = 50000
test_all =  10000

#normalize
transform = transforms.Compose(
    [transforms.ToTensor(),
     ])
    #transforms.Normalize((0.49139968, 0.48215827 ,0.44653124), (0.24703233 0.24348505 0.26158768))
#train
cifar10_train_dataset = datasets.CIFAR10(root='./', # here
                               train=True, # train split
                               download=True, # we want to get the data
                               transform=transform #put it into tensor format
                        )
train_data = torch.utils.data.DataLoader(cifar10_train_dataset,
                        batch_size=train_all,
                        shuffle=True
                        )

#test
cifar10_test_dataset = datasets.CIFAR10(root='./', # here
                               train=False, # test split
                               download=True, # we want to get the data
                               transform=transform # put it into tensor format
                        )
test_data = torch.utils.data.DataLoader(cifar10_test_dataset,
                        batch_size=test_all,
                        shuffle=True
                        )

Files already downloaded and verified
Files already downloaded and verified


In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# parameters
learning_rate=5e-1
nepochs = 10
nout=10

LossFunction = nn.CrossEntropyLoss()

In [9]:
k_percents = []
for i in range(9):
    #want decreasing percents 90%, 80%, ..., 10%
    factor = 0.9 - (i*0.1)
    k_percents.append(round(3072*factor))
print(k_percents)
# welp, large %'s dont fit in memory, so I just use higher indices below

[2765, 2458, 2150, 1843, 1536, 1229, 922, 614, 307]


In [10]:
elapsed = []
loss_histories = []
accuracies = []
for i in range(6,9):
    k = k_percents[i]
    ninputs=k
    nhidden=k
    model = SimpleNet(ninputs=ninputs,
                  nhidden=nhidden,
                  nout=nout).to(device)
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    e, l, a = trainAndTest_PCA(k, train_data, test_data, model)
    elapsed.append(e)
    loss_histories.append(l)
    accuracies.append(a)

Performing PCA with k value: 922
Training Start


Epoch:  10%|█         | 1/10 [00:09<01:28,  9.79s/epoch]

Epoch 0: loss: 2.302915096282959


Epoch:  20%|██        | 2/10 [00:20<01:24, 10.62s/epoch]

Epoch 1: loss: 2.302809953689575


...

Epoch:  90%|█████████ | 9/10 [01:18<00:08,  8.77s/epoch]

Epoch 8: loss: 2.3028879165649414


Epoch: 100%|██████████| 10/10 [01:27<00:00,  8.71s/epoch]

Epoch 9: loss: 2.3026909828186035
Test Start


Total accuracy = 9.38%


In [11]:
print(elapsed)

[85.47521591186523, 82.75961494445801, 87.05348753929138]


In [12]:
print(loss_histories)

[[2.302915096282959, 2.302809953689575, 2.3029322624206543, 2.3028581142425537, 2.3027565479278564, 2.302609443664551, 2.3028182983398438, 2.3027164936065674, 2.3027055263519287, 2.3026790618896484], [2.3029913902282715, 2.3029422760009766, 2.3027777671813965, 2.3027191162109375, 2.302753210067749, 2.302872896194458, 2.3029701709747314, 2.302804708480835, 2.302586317062378, 2.302849292755127], [2.303600311279297, 2.303173065185547, 2.303170680999756, 2.3029794692993164, 2.3031063079833984, 2.3028643131256104, 2.302830219268799, 2.302980899810791, 2.3028879165649414, 2.3026909828186035]]


In [13]:
for i in range(len(accuracies)):
    print(f"Value of k: {k_percents[i]}, accuracy: {accuracies[i].item()}")

9.899999618530273
9.989999771118164
9.380000114440918
